# Rough Work Model tuning and testing was done in Python notebook. Used this notebook for reference and Rough Work

In [2]:
# Set environment params
Sys.setenv(LANG='en')  # English

# Import libraries
library(data.table)  # Data manipulate
library(mlr)         # ML toolkit
library(caret)       # ML toolkit
library(ggplot2)     # Visualization
library(pROC)        # AUC, ROC
library(e1071)       # Evaluation
library(gridExtra)   # Visualization
library(kknn)        # kNN model
library(nnet)        # One-vs-All Logistic Regression
library(dummies)     # Data processing
library(readr)
library(tidyverse)

# Resize plot
library(repr)  # String and binary representations
#options(repr.plot.width=4, repr.plot.height=4)

R.version.string

Loading required package: ParamHelpers

Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: ggplot2

Loading required package: lattice


Attaching package: 'caret'


The following object is masked from 'package:mlr':

    train


Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following objects are masked from 'package:stats':

    cov, smooth, var



Attaching package: 'e1071'


The following object is masked from 'package:mlr':

    impute



Attaching package: 'kknn'


The following object is masked from 'package:caret':

    contr.dummy


dummies-1.5.6 provided by Decision Patterns


-- Attaching packages --------------------------------------------------------------------- tidyverse 1.3.1 --

v tibble  3.1.6     v dplyr   1.0.7
v tidy

[1] "R version 4.0.5 (2021-03-31)"

In [3]:
X_train =  read_csv('./Data/X_train.csv')
X_test =  read_csv('./Data/X_test.csv')
y_train =  read_csv('./Data/y_train.csv')
y_test =  read_csv('./Data/y_test.csv')

New names:
* `` -> ...1

Rows: 16000 Columns: 84

-- Column specification --------------------------------------------------------------------------------------
Delimiter: ","
dbl (84): ...1, client_id, campaign, pdays, previous, emp.var.rate, cons.pri...


i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.

New names:
* `` -> ...1

Rows: 4000 Columns: 84

-- Column specification --------------------------------------------------------------------------------------
Delimiter: ","
dbl (84): ...1, client_id, campaign, pdays, previous, emp.var.rate, cons.pri...


i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.

New names:
* `` -> ...1

Rows: 16000 Columns: 2

-- Column specification --------------------------------------------------------------------------------------
Delimiter: ","


In [96]:
train = cbind(X_train, y_train)
test = cbind(X_test, y_test)

In [97]:
colnames(train) <- gsub("_", ".", colnames(train))
colnames(train) <- gsub("-", ".", colnames(train))
train_features$subscribe <- as.factor(train_features$subscribe)
colnames(X_test) <- gsub("_", ".", colnames(X_test))
colnames(X_test) <- gsub("-", ".", colnames(X_test))

In [98]:
train$...1 <- NULL

In [99]:
train$...1 <-NULL

In [100]:
str(train)

'data.frame':	16000 obs. of  84 variables:
 $ client.id                    : num  22764 38348 6856 171 20827 ...
 $ campaign                     : num  1 1 1 2 2 2 4 3 5 1 ...
 $ pdays                        : num  999 999 999 999 999 999 999 999 999 999 ...
 $ previous                     : num  2 0 0 0 0 0 0 0 0 0 ...
 $ emp.var.rate                 : num  -2.9 -1.7 1.4 -1.8 -1.8 1.1 1.4 1.4 1.4 1.4 ...
 $ cons.price.idx               : num  92.2 94.1 93.9 92.9 93.1 ...
 $ cons.conf.idx                : num  -31.4 -39.8 -42.7 -46.2 -47.1 -36.4 -41.8 -36.1 -42.7 -41.8 ...
 $ euribor3m                    : num  0.843 0.729 4.961 1.266 1.445 ...
 $ nr.employed                  : num  5076 4992 5228 5099 5099 ...
 $ job.admin.                   : num  1 0 1 0 1 1 0 0 1 0 ...
 $ job.blue.collar              : num  0 0 0 1 0 0 1 0 0 0 ...
 $ job.entrepreneur             : num  0 0 0 0 0 0 0 0 0 1 ...
 $ job.housemaid                : num  0 0 0 0 0 0 0 0 0 0 ...
 $ job.management          

In [101]:
test$...1 <- NULL
test$...1 <-NULL

In [103]:
str(X_test)

spec_tbl_df [4,000 x 84] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ ...1                         : num [1:4000] 10650 2041 8668 1114 13902 ...
 $ client.id                    : num [1:4000] 16820 3734 12920 25110 28742 ...
 $ campaign                     : num [1:4000] 4 2 2 2 1 4 2 6 2 7 ...
 $ pdays                        : num [1:4000] 999 999 999 999 999 999 999 999 999 999 ...
 $ previous                     : num [1:4000] 0 0 0 0 0 0 0 0 0 0 ...
 $ emp.var.rate                 : num [1:4000] 1.1 -1.1 1.4 -0.1 -0.1 1.4 -3.4 -1.8 1.4 1.4 ...
 $ cons.price.idx               : num [1:4000] 94 94.8 94.5 93.2 93.2 ...
 $ cons.conf.idx                : num [1:4000] -36.4 -50.8 -41.8 -42 -42 -36.1 -26.9 -46.2 -36.1 -42.7 ...
 $ euribor3m                    : num [1:4000] 4.86 1.04 4.96 4.12 4.15 ...
 $ nr.employed                  : num [1:4000] 5191 4964 5228 5196 5196 ...
 $ job.admin.                   : num [1:4000] 0 0 0 0 1 0 0 0 0 0 ...
 $ job.blue.collar              : num [1:400

In [104]:
# Fit Logistic Regression

# Fit Logistic Regression model
md_logreg <- glm(subscribe ∼ .,
                 data=train, family=binomial)
summary(md_logreg)


Call:
glm(formula = subscribe ~ ., family = binomial, data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2209  -0.4020  -0.3166  -0.2472   3.0181  

Coefficients: (11 not defined because of singularities)
                                Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -5.691e+01  1.412e+02  -0.403 0.686894    
client.id                      6.419e-07  2.402e-06   0.267 0.789269    
campaign                      -4.265e-02  1.455e-02  -2.932 0.003368 ** 
pdays                         -1.190e-03  2.986e-04  -3.986 6.73e-05 ***
previous                       6.581e-03  8.677e-02   0.076 0.939543    
emp.var.rate                  -3.906e-01  1.119e-01  -3.490 0.000482 ***
cons.price.idx                 6.372e-01  1.807e-01   3.526 0.000422 ***
cons.conf.idx                  2.886e-02  8.574e-03   3.366 0.000763 ***
euribor3m                     -1.574e-01  1.026e-01  -1.534 0.125037    
nr.employed                   

In [105]:
# Make prediction on both train and test data
pred_train_prob <- predict(md_logreg, newdata=train, type='response')
head(pred_train_prob)

pred_test_prob <- predict(md_logreg, newdata=X_test, type='response')
head(pred_test_prob)

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"


1          2          3          4          5          6 
0.15536554 0.36804920 0.07249783 0.10070372 0.20281176 0.03676632

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"


1          2          3          4          5          6 
0.02769367 0.28188812 0.08144113 0.07636650 0.04152872 0.06092750

In [106]:
# Evaluation model on train
threshold <- 0.11355  # Decision threshold
pred_train <- rep(0, length(pred_train_prob))
pred_train[pred_train_prob > threshold] <- 1

# Confusion matrix
print('Confusion matrix:')
table(pred_train, train$subscribe)

# Accuracy and error rate
acc <- mean(pred_train == train$subscribe)
err_rate <- 1 - acc
print(paste('Accuracy =', round(acc, 2), '%'))
print(paste('Error =', round(err_rate, 2), '%'))

# AUC
pred_roc <- pROC::roc(as.numeric(pred_train_prob), as.numeric(train$subscribe == 1))
pred_auc <- pROC::auc(pred_roc)
print(paste('AUC =', pred_auc))

[1] "Confusion matrix:"


          
pred_train     0     1
         0 11945   636
         1  2268  1151

[1] "Accuracy = 0.82 %"
[1] "Error = 0.18 %"


Warning message in roc.default(as.numeric(pred_train_prob), as.numeric(train$subscribe == :
"'response' has more than two levels. Consider setting 'levels' explicitly or using 'multiclass.roc' instead"
Setting levels: control = 7.67637686915498e-06, case = 1.14428776304388e-05

Setting direction: controls < cases



[1] "AUC = 0.5"


In [107]:
# Evaluation model on test
threshold <- 0.11355  # Decision threshold
pred_test <- rep(0, length(pred_test_prob))
pred_test[pred_test_prob > threshold] <- 1

# Confusion matrix
print('Confusion matrix:')
table(pred_test, test$subscribe)

# Accuracy and error rate
acc <- mean(pred_test == test$subscribe)
err_rate <- 1 - acc
print(paste('Accuracy =', round(acc, 2), '%'))
print(paste('Error =', round(err_rate, 2), '%'))

# AUC
pred_roc <- pROC::roc(as.numeric(pred_test_prob), as.numeric(test$subscribe == 1))
pred_auc <- pROC::auc(pred_roc)
print(paste('AUC =', pred_auc))

[1] "Confusion matrix:"


         
pred_test    0    1
        0 2934  195
        1  582  289

[1] "Accuracy = 0.81 %"
[1] "Error = 0.19 %"


Warning message in roc.default(as.numeric(pred_test_prob), as.numeric(test$subscribe == :
"'response' has more than two levels. Consider setting 'levels' explicitly or using 'multiclass.roc' instead"
Setting levels: control = 0.00720045600916906, case = 0.00857747990418644

Setting direction: controls < cases



[1] "AUC = 0.5"


In [14]:
library(Boruta)

In [15]:
# Perform Boruta search
boruta_output <- Boruta(subscribe ~ ., data=train, doTrace=0) 

Computing permutation importance.. Progress: 46%. Estimated remaining time: 35 seconds.
Computing permutation importance.. Progress: 91%. Estimated remaining time: 5 seconds.
Computing permutation importance.. Progress: 46%. Estimated remaining time: 36 seconds.
Computing permutation importance.. Progress: 91%. Estimated remaining time: 6 seconds.
Computing permutation importance.. Progress: 44%. Estimated remaining time: 39 seconds.
Computing permutation importance.. Progress: 88%. Estimated remaining time: 8 seconds.
Computing permutation importance.. Progress: 42%. Estimated remaining time: 42 seconds.
Computing permutation importance.. Progress: 81%. Estimated remaining time: 14 seconds.
Computing permutation importance.. Progress: 48%. Estimated remaining time: 33 seconds.
Computing permutation importance.. Progress: 92%. Estimated remaining time: 5 seconds.
Computing permutation importance.. Progress: 45%. Estimated remaining time: 38 seconds.
Computing permutation importance.. P

In [16]:
boruta_signif <- getSelectedAttributes(boruta_output, withTentative = TRUE)
 


In [17]:
print(boruta_signif[1:20]) 

 [1] "campaign"                    "pdays"                      
 [3] "previous"                    "emp.var.rate"               
 [5] "cons.price.idx"              "cons.conf.idx"              
 [7] "euribor3m"                   "nr.employed"                
 [9] "`job_blue-collar`"           "job_retired"                
[11] "job_student"                 "job_unknown"                
[13] "marital_married"             "marital_single"             
[15] "education_university.degree" "default_no"                 
[17] "default_unknown"             "housing_no"                 
[19] "housing_yes"                 "contact_cellular"           


In [39]:
col.num <- which(colnames(train) %in% boruta_signif[1:5])
train_features <- train[,col.num]
train_features <- cbind(train_features, train['subscribe']) 
str(train_features)

'data.frame':	16000 obs. of  6 variables:
 $ campaign      : num  1 1 1 2 2 2 4 3 5 1 ...
 $ pdays         : num  999 999 999 999 999 999 999 999 999 999 ...
 $ previous      : num  2 0 0 0 0 0 0 0 0 0 ...
 $ emp.var.rate  : num  -2.9 -1.7 1.4 -1.8 -1.8 1.1 1.4 1.4 1.4 1.4 ...
 $ cons.price.idx: num  92.2 94.1 93.9 92.9 93.1 ...
 $ subscribe     : num  0 0 0 0 1 0 0 0 0 0 ...


In [40]:
# Fit Logistic Regression

# Fit Logistic Regression model
md_logreg <- glm(subscribe ∼ .,
                 data=train_features, family=binomial)
summary(md_logreg)


Call:
glm(formula = subscribe ~ ., family = binomial, data = train_features)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.8502  -0.4816  -0.4816  -0.4816   2.1527  

Coefficients:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -1.989e+00  2.823e-02 -70.460  < 2e-16 ***
campaign       -9.658e-06  2.461e-05  -0.392    0.695    
pdays          -1.062e-04  1.635e-05  -6.495 8.28e-11 ***
previous        1.173e-05  2.655e-05   0.442    0.659    
emp.var.rate   -7.112e-06  2.659e-05  -0.267    0.789    
cons.price.idx -8.615e-06  2.507e-05  -0.344    0.731    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 11201  on 15999  degrees of freedom
Residual deviance: 11165  on 15994  degrees of freedom
AIC: 11177

Number of Fisher Scoring iterations: 4


In [41]:
# Make prediction on both train and test data
pred_train_prob <- predict(md_logreg, newdata=train, type='response')
head(pred_train_prob)

pred_test_prob <- predict(md_logreg, newdata=test, type='response')
head(pred_test_prob)

1         2         3         4         5         6 
0.1094930 0.1094884 0.1094863 0.1094885 0.1094883 0.1094855

1         2         3         4         5         6 
0.1094837 0.1094864 0.1094849 0.1094870 0.1094880 0.1094839

In [42]:
# Evaluation model on train
threshold <- 0.11355  # Decision threshold
pred_train <- rep(0, length(pred_train_prob))
pred_train[pred_train_prob > threshold] <- 1

# Confusion matrix
print('Confusion matrix:')
table(pred_train, train$subscribe)

# Accuracy and error rate
acc <- mean(pred_train == train$subscribe)
err_rate <- 1 - acc
print(paste('Accuracy =', round(acc, 2), '%'))
print(paste('Error =', round(err_rate, 2), '%'))

# AUC
pred_roc <- pROC::roc(as.numeric(pred_train_prob), as.numeric(train$subscribe == 1))
pred_auc <- pROC::auc(pred_roc)
print(paste('AUC =', pred_auc))

[1] "Confusion matrix:"


          
pred_train     0     1
         0 13505  1349
         1   708   438

[1] "Accuracy = 0.87 %"
[1] "Error = 0.13 %"


Warning message in roc.default(as.numeric(pred_train_prob), as.numeric(train$subscribe == :
"'response' has more than two levels. Consider setting 'levels' explicitly or using 'multiclass.roc' instead"
Setting levels: control = 0.098544822535546, case = 0.0985589906606137

Setting direction: controls < cases



[1] "AUC = 0.5"


In [44]:
# Evaluation model on test
threshold <- 0.11355  # Decision threshold
pred_test <- rep(0, length(pred_test_prob))
pred_test[pred_test_prob > threshold] <- 1

# Confusion matrix
print('Confusion matrix:')
table(pred_test, test$subscribe)

# Accuracy and error rate
acc <- mean(pred_test == test$subscribe)
err_rate <- 1 - acc
print(paste('Accuracy =', round(acc, 2), '%'))
print(paste('Error =', round(err_rate, 2), '%'))

# AUC
pred_roc <- pROC::roc(pred_test_prob, test$subscribe)
pred_auc <- pROC::auc(pred_roc)
print(paste('AUC =', pred_auc))

[1] "Confusion matrix:"


         
pred_test    0    1
        0 3335  376
        1  181  108

[1] "Accuracy = 0.86 %"
[1] "Error = 0.14 %"


Warning message in roc.default(pred_test_prob, test$subscribe):
"'response' has more than two levels. Consider setting 'levels' explicitly or using 'multiclass.roc' instead"
Setting levels: control = 0.0985537655224669, case = 0.0985571978097783

Setting direction: controls < cases



[1] "AUC = 0.5"


In [23]:
library(glmnet)

x <- as.matrix(X_train) # all X vars
y <- as.matrix( y_train) # Only Class

# Fit the LASSO model (Lasso: Alpha = 1)
set.seed(100)
cv.lasso <- cv.glmnet(x, y, family='binomial', alpha=1, parallel=TRUE, standardize=TRUE, type.measure='auc')


Loading required package: Matrix


Attaching package: 'Matrix'


The following objects are masked from 'package:tidyr':

    expand, pack, unpack


Loaded glmnet 4.1-2

Warning message:
"executing %dopar% sequentially: no parallel backend registered"


In [24]:
df_coef <- round(as.matrix(coef(cv.lasso, s=cv.lasso$lambda.min)), 2)

# See all contributing variables
df_coef[df_coef[, 1] != 0, ]

(Intercept)                   nr.employed 
                        32.11                         -0.01 
              job_blue-collar              job_entrepreneur 
                        -0.19                          0.02 
                job_housemaid                job_management 
                        -0.03                          0.08 
                  job_retired             job_self-employed 
                        -0.02                         -0.18 
                 job_services                   job_student 
                        -0.02                          0.12 
               job_technician                job_unemployed 
                         0.03                          0.13 
                  job_unknown              marital_divorced 
                         0.01                         -0.06 
               marital_single               marital_unknown 
                         0.13                          0.34 
           education_basic.4y            education_basic.9y 
                        -0.05                          0.02 
         education_illiterate education_professional.course 
                         0.52                         -0.01 
  education_university.degree             education_unknown 
                         0.07                          0.05 
                   default_no                   default_yes 
                         0.31                         -3.08 
              housing_unknown                   housing_yes 
                         0.10                         -0.02 
                 loan_unknown                      loan_yes 
                        -0.11                          0.04 
             contact_cellular             contact_telephone 
                         0.06                         -0.34 
                    month_apr                     month_dec 
                         0.05                          0.20 
                    month_jul                     month_jun 
                         0.14                          0.21 
                    month_mar                     month_may 
                         0.43                         -0.47 
                    month_nov                     month_oct 
                        -0.20                         -0.05 
                    month_sep                 month_unknown 
                        -0.20                         -0.33 
              day_of_week_fri               day_of_week_mon 
                        -0.01                         -0.18 
              day_of_week_tue           day_of_week_unknown 
                         0.01                         -0.10 
              day_of_week_wed              poutcome_failure 
                         0.06                         -0.34 
             poutcome_success              poutcome_unknown 
                         0.12                         -0.21 
              age-group_10-20               age-group_30-40 
                         0.07                         -0.09 
              age-group_40-50               age-group_50-60 
                        -0.07                          0.05 
              age-group_60-70               age-group_70-80 
                         0.25                          0.27 
              age-group_80-90              age-group_90-100 
                         0.27                          0.35 
                pdays_missing              previous_missing 
                        -5.12                         -0.19 
          nr.employed_missing 
                      -123.01

In [25]:
# Step 1: Define base intercept only model
base.mod <- glm(subscribe ~ 1 , data=train, family=binomial)  

# Step 2: Full model with all predictors
all.mod <- glm(subscribe ~ . , data= train, family=binomial) 

# Step 3: Perform step-wise algorithm. direction='both' implies both forward and backward stepwise
stepMod <- step(base.mod, scope = list(lower = base.mod, upper = all.mod), direction = "both", trace = 0, steps = 1000)  

# Step 4: Get the shortlisted variable.
shortlistedVars <- names(unlist(stepMod[[1]])) 
shortlistedVars <- shortlistedVars[!shortlistedVars %in% "(Intercept)"] # remove intercept

# Show
print(shortlistedVars)

 [1] "poutcome_success"            "contact_telephone"          
 [3] "month_mar"                   "month_sep"                  
 [5] "default_no"                  "month_jun"                  
 [7] "`age-group_60-70`"           "job_student"                
 [9] "`age-group_70-80`"           "marital_single"             
[11] "`job_blue-collar`"           "day_of_week_mon"            
[13] "marital_unknown"             "`age-group_30-40`"          
[15] "education_university.degree" "`job_self-employed`"        
[17] "nr.employed_missing"         "nr.employed"                
[19] "month_may"                   "poutcome_failure"           
[21] "month_jul"                   "month_nov"                  
[23] "day_of_week_wed"            


In [53]:
col.num <- which(colnames(train) %in% shortlistedVars)
train_features <- train[,col.num]
train_features <- cbind(train_features, train['subscribe']) 
str(train_features)

'data.frame':	16000 obs. of  19 variables:
 $ nr.employed                : num  5076 4992 5228 5099 5099 ...
 $ job_student                : num  0 0 0 0 0 0 0 0 0 0 ...
 $ marital_single             : num  0 0 0 0 0 1 0 0 0 0 ...
 $ marital_unknown            : num  0 0 0 0 0 0 0 0 0 0 ...
 $ education_university.degree: num  1 0 0 0 1 0 0 0 1 0 ...
 $ default_no                 : num  1 1 1 1 1 1 1 0 1 0 ...
 $ contact_telephone          : num  0 1 0 0 0 1 1 0 0 1 ...
 $ month_jul                  : num  0 0 1 0 0 0 0 0 1 0 ...
 $ month_jun                  : num  0 1 0 0 0 0 1 0 0 1 ...
 $ month_mar                  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ month_may                  : num  0 0 0 1 0 1 0 0 0 0 ...
 $ month_nov                  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ month_sep                  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ day_of_week_mon            : num  1 0 0 0 0 0 1 0 1 0 ...
 $ day_of_week_wed            : num  0 0 0 0 1 0 0 0 0 0 ...
 $ poutcome_failure           : num  

In [54]:
# Fit Logistic Regression

# Fit Logistic Regression model
md_logreg <- glm(subscribe ∼ .,
                 data=train_features, family=binomial)
summary(md_logreg)


Call:
glm(formula = subscribe ~ ., family = binomial, data = train_features)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1865  -0.4030  -0.3235  -0.2509   2.8810  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  5.698e+01  2.222e+00  25.648  < 2e-16 ***
nr.employed                 -1.152e-02  4.310e-04 -26.728  < 2e-16 ***
job_student                  3.308e-01  1.563e-01   2.116 0.034316 *  
marital_single               1.402e-01  6.359e-02   2.204 0.027504 *  
marital_unknown              5.911e-01  2.198e-01   2.689 0.007161 ** 
education_university.degree  1.445e-01  6.107e-02   2.366 0.018001 *  
default_no                   3.208e-01  8.879e-02   3.613 0.000303 ***
contact_telephone           -3.861e-01  7.895e-02  -4.891 1.00e-06 ***
month_jul                    3.140e-01  9.285e-02   3.382 0.000721 ***
month_jun                    3.027e-01  1.022e-01   2.961 0.003064 ** 
month_mar   

In [55]:
# Make prediction on both train and test data
pred_train_prob <- predict(md_logreg, newdata=train, type='response')
head(pred_train_prob)

pred_test_prob <- predict(md_logreg, newdata=test, type='response')
head(pred_test_prob)

1          2          3          4          5          6 
0.14807240 0.42977760 0.06850481 0.10872441 0.23367348 0.03203212

1          2          3          4          5          6 
0.02795959 0.41417774 0.05944108 0.07538355 0.04749402 0.05844852

In [56]:
# Evaluation model on train
threshold <- 0.11355  # Decision threshold
pred_train <- rep(0, length(pred_train_prob))
pred_train[pred_train_prob > threshold] <- 1

# Confusion matrix
print('Confusion matrix:')
table(pred_train, train$subscribe)

# Accuracy and error rate
acc <- mean(pred_train == train$subscribe)
err_rate <- 1 - acc
print(paste('Accuracy =', round(acc, 2), '%'))
print(paste('Error =', round(err_rate, 2), '%'))

# AUC
pred_roc <- pROC::roc(as.numeric(pred_train_prob), as.numeric(train$subscribe == 1))
pred_auc <- pROC::auc(pred_roc)
print(paste('AUC =', pred_auc))

[1] "Confusion matrix:"


          
pred_train     0     1
         0 11792   642
         1  2421  1145

[1] "Accuracy = 0.81 %"
[1] "Error = 0.19 %"


Warning message in roc.default(as.numeric(pred_train_prob), as.numeric(train$subscribe == :
"'response' has more than two levels. Consider setting 'levels' explicitly or using 'multiclass.roc' instead"
Setting levels: control = 0.0157621027654653, case = 0.0179530614605351

Setting direction: controls < cases



[1] "AUC = 0.48780487804878"


In [57]:
# Evaluation model on test
threshold <- 0.11355  # Decision threshold
pred_test <- rep(0, length(pred_test_prob))
pred_test[pred_test_prob > threshold] <- 1

# Confusion matrix
print('Confusion matrix:')
table(pred_test, test$subscribe)

# Accuracy and error rate
acc <- mean(pred_test == test$subscribe)
err_rate <- 1 - acc
print(paste('Accuracy =', round(acc, 2), '%'))
print(paste('Error =', round(err_rate, 2), '%'))

# AUC
pred_roc <- pROC::roc(pred_test_prob, test$subscribe)
pred_auc <- pROC::auc(pred_roc)
print(paste('AUC =', pred_auc))

[1] "Confusion matrix:"


         
pred_test    0    1
        0 2914  193
        1  602  291

[1] "Accuracy = 0.8 %"
[1] "Error = 0.2 %"


Warning message in roc.default(pred_test_prob, test$subscribe):
"'response' has more than two levels. Consider setting 'levels' explicitly or using 'multiclass.roc' instead"
Setting levels: control = 0.0157621027654653, case = 0.0180910437411432

Setting direction: controls < cases



[1] "AUC = 0.484848484848485"


In [66]:
make.names(names(train))
names(train)[names(train) == 'job_blue.collar'] <- 'job.blue.collar'

[1] "client_id"                     "campaign"                     
 [3] "pdays"                         "previous"                     
 [5] "emp.var.rate"                  "cons.price.idx"               
 [7] "cons.conf.idx"                 "euribor3m"                    
 [9] "nr.employed"                   "job_admin."                   
[11] "job_blue.collar"               "job_entrepreneur"             
[13] "job_housemaid"                 "job_management"               
[15] "job_retired"                   "job_self.employed"            
[17] "job_services"                  "job_student"                  
[19] "job_technician"                "job_unemployed"               
[21] "job_unknown"                   "marital_divorced"             
[23] "marital_married"               "marital_single"               
[25] "marital_unknown"               "education_basic.4y"           
[27] "education_basic.6y"            "education_basic.9y"           
[29] "education_high.school"         "education_illiterate"         
[31] "education_professional.course" "education_university.degree"  
[33] "education_unknown"             "default_no"                   
[35] "default_unknown"               "default_yes"                  
[37] "housing_no"                    "housing_unknown"              
[39] "housing_yes"                   "loan_no"                      
[41] "loan_unknown"                  "loan_yes"                     
[43] "contact_cellular"              "contact_telephone"            
[45] "contact_unknown"               "month_apr"                    
[47] "month_aug"                     "month_dec"                    
[49] "month_jul"                     "month_jun"                    
[51] "month_mar"                     "month_may"                    
[53] "month_nov"                     "month_oct"                    
[55] "month_sep"                     "month_unknown"                
[57] "day_of_week_fri"               "day_of_week_mon"              
[59] "day_of_week_thu"               "day_of_week_tue"              
[61] "day_of_week_unknown"           "day_of_week_wed"              
[63] "poutcome_failure"              "poutcome_nonexistent"         
[65] "poutcome_success"              "poutcome_unknown"             
[67] "age.group_10.20"               "age.group_20.30"              
[69] "age.group_30.40"               "age.group_40.50"              
[71] "age.group_50.60"               "age.group_60.70"              
[73] "age.group_70.80"               "age.group_80.90"              
[75] "age.group_90.100"              "campaign_missing"             
[77] "pdays_missing"                 "previous_missing"             
[79] "emp.var.rate_missing"          "cons.price.idx_missing"       
[81] "cons.conf.idx_missing"         "euribor3m_missing"            
[83] "nr.employed_missing"           "subscribe"

In [85]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=10)

# Define the model
lnr_logreg <- makeLearner("classif.logreg", predict.type="prob", fix.factors.prediction=T)

# Define the model
task_logreg <- makeClassifTask(id="student", data=train, target="subscribe")

# Run the k-fold CV
resample(lnr_logreg, task_logreg, rdesc, measures=list(mlr::auc))

Resampling: cross-validation

Measures:             auc       

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
[Resample] iter 1:    0.7721610 

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
[Resample] iter 2:    0.7725399 

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
[Resample] iter 3:    0.8015194 

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
[Resample] iter 4:    0.7606314 

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
[Resample] iter 5:    0.7834091 

Warning message in predict.lm(object, newdata,

Resample Result
Task: student
Learner: classif.logreg
Aggr perf: auc.test.mean=0.7861178
Runtime: 11.5901

In [88]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=10)

# Define the model
lnr_logreg <- makeLearner("classif.logreg", predict.type="prob", fix.factors.prediction=T)

# Define the model
task_logreg <- makeClassifTask(id="student", data=train_features, target="subscribe")

# Run the k-fold CV
resample(lnr_logreg, task_logreg, rdesc, measures=list(mlr::auc))

Resampling: cross-validation

Measures:             auc       

[Resample] iter 1:    0.8064829 

[Resample] iter 2:    0.7979945 

[Resample] iter 3:    0.7696111 

[Resample] iter 4:    0.7685906 

[Resample] iter 5:    0.7688388 

[Resample] iter 6:    0.8027884 

[Resample] iter 7:    0.8151812 

[Resample] iter 8:    0.7786420 

[Resample] iter 9:    0.8001708 

[Resample] iter 10:   0.8079904 




Aggregated Result: auc.test.mean=0.7916291






Resample Result
Task: student
Learner: classif.logreg
Aggr perf: auc.test.mean=0.7916291
Runtime: 1.51214

In [59]:
# Fit the KNN model with the train set using 2 best predictors
library(class)

# k = 1
set.seed(123)
knn1_pred <- knn(train=train,
                 test=test,
                 cl=train$subscribe,
                 k=1)

# Confusion matrix
print('Confusion matrix:')
table(knn1_pred, test$subscribe)

# Accuracy
acc <- mean(knn1_pred == test$subscribe)
print(paste('Accuracy =', round(acc, 2), '%'))

[1] "Confusion matrix:"


         
knn1_pred    0    1
        0 3213  350
        1  303  134

[1] "Accuracy = 0.84 %"


In [71]:
# k = 3
set.seed(123)
knn3_pred <- knn(train=train,
                 test=test,
                 cl=train$subscribe,
                 k=3)

# Confusion matrix
print('Confusion matrix:')
table(knn3_pred, test$subscribe)

# Accuracy
acc <- mean(knn3_pred == test$subscribe)
print(paste('Accuracy =', round(acc, 2), '%'))

[1] "Confusion matrix:"


         
knn3_pred    0    1
        0 3363  378
        1  153  106

[1] "Accuracy = 0.87 %"


In [83]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=10)

# Define the model
lnr_knn <- makeLearner("classif.kknn", predict.type="prob", fix.factors.prediction=T, k=1)

# Define the task
task_logreg <- makeClassifTask(id="student", data=train, target="subscribe")

# Run the k-fold CV
resample(lnr_knn, task_logreg, rdesc, measures=list(mlr::acc))

Resampling: cross-validation

Measures:             acc       

[Resample] iter 1:    0.8431250 

[Resample] iter 2:    0.8493750 

[Resample] iter 3:    0.8443750 

[Resample] iter 4:    0.8531250 

[Resample] iter 5:    0.8193750 

[Resample] iter 6:    0.8375000 

[Resample] iter 7:    0.8443750 

[Resample] iter 8:    0.8481250 

[Resample] iter 9:    0.8493750 

[Resample] iter 10:   0.8468750 




Aggregated Result: acc.test.mean=0.8435625






Resample Result
Task: student
Learner: classif.kknn
Aggr perf: acc.test.mean=0.8435625
Runtime: 21.7467

In [80]:
names(train)

[1] "client.id"                     "campaign"                     
 [3] "pdays"                         "previous"                     
 [5] "emp.var.rate"                  "cons.price.idx"               
 [7] "cons.conf.idx"                 "euribor3m"                    
 [9] "nr.employed"                   "job.admin."                   
[11] "job.blue.collar"               "job.entrepreneur"             
[13] "job.housemaid"                 "job.management"               
[15] "job.retired"                   "job.self.employed"            
[17] "job.services"                  "job.student"                  
[19] "job.technician"                "job.unemployed"               
[21] "job.unknown"                   "marital.divorced"             
[23] "marital.married"               "marital.single"               
[25] "marital.unknown"               "education.basic.4y"           
[27] "education.basic.6y"            "education.basic.9y"           
[29] "education.high.school"         "education.illiterate"         
[31] "education.professional.course" "education.university.degree"  
[33] "education.unknown"             "default.no"                   
[35] "default.unknown"               "default.yes"                  
[37] "housing.no"                    "housing.unknown"              
[39] "housing.yes"                   "loan.no"                      
[41] "loan.unknown"                  "loan.yes"                     
[43] "contact.cellular"              "contact.telephone"            
[45] "contact.unknown"               "month.apr"                    
[47] "month.aug"                     "month.dec"                    
[49] "month.jul"                     "month.jun"                    
[51] "month.mar"                     "month.may"                    
[53] "month.nov"                     "month.oct"                    
[55] "month.sep"                     "month.unknown"                
[57] "day.of.week.fri"               "day.of.week.mon"              
[59] "day.of.week.thu"               "day.of.week.tue"              
[61] "day.of.week.unknown"           "day.of.week.wed"              
[63] "poutcome.failure"              "poutcome.nonexistent"         
[65] "poutcome.success"              "poutcome.unknown"             
[67] "age.group.10.20"               "age.group.20.30"              
[69] "age.group.30.40"               "age.group.40.50"              
[71] "age.group.50.60"               "age.group.60.70"              
[73] "age.group.70.80"               "age.group.80.90"              
[75] "age.group.90.100"              "campaign.missing"             
[77] "pdays.missing"                 "previous.missing"             
[79] "emp.var.rate.missing"          "cons.price.idx.missing"       
[81] "cons.conf.idx.missing"         "euribor3m.missing"            
[83] "nr.employed.missing"           "subscribe"

In [84]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=10)

# Define the model
lnr_knn <- makeLearner("classif.kknn", predict.type="prob", fix.factors.prediction=T, k=3)

# Define the task
task_logreg <- makeClassifTask(id="student", data=train, target="subscribe")

# Run the k-fold CV
resample(lnr_knn, task_logreg, rdesc, measures=list(mlr::acc))

Resampling: cross-validation

Measures:             acc       

[Resample] iter 1:    0.8481250 

[Resample] iter 2:    0.8437500 

[Resample] iter 3:    0.8293750 

[Resample] iter 4:    0.8412500 

[Resample] iter 5:    0.8350000 

[Resample] iter 6:    0.8431250 

[Resample] iter 7:    0.8431250 

[Resample] iter 8:    0.8468750 

[Resample] iter 9:    0.8418750 

[Resample] iter 10:   0.8412500 




Aggregated Result: acc.test.mean=0.8413750






Resample Result
Task: student
Learner: classif.kknn
Aggr perf: acc.test.mean=0.8413750
Runtime: 27.5089

In [112]:
str(X_train)

spec_tbl_df [16,000 x 84] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ ...1                         : num [1:16000] 5894 3728 8958 7671 5999 ...
 $ client_id                    : num [1:16000] 22764 38348 6856 171 20827 ...
 $ campaign                     : num [1:16000] 1 1 1 2 2 2 4 3 5 1 ...
 $ pdays                        : num [1:16000] 999 999 999 999 999 999 999 999 999 999 ...
 $ previous                     : num [1:16000] 2 0 0 0 0 0 0 0 0 0 ...
 $ emp.var.rate                 : num [1:16000] -2.9 -1.7 1.4 -1.8 -1.8 1.1 1.4 1.4 1.4 1.4 ...
 $ cons.price.idx               : num [1:16000] 92.2 94.1 93.9 92.9 93.1 ...
 $ cons.conf.idx                : num [1:16000] -31.4 -39.8 -42.7 -46.2 -47.1 -36.4 -41.8 -36.1 -42.7 -41.8 ...
 $ euribor3m                    : num [1:16000] 0.843 0.729 4.961 1.266 1.445 ...
 $ nr.employed                  : num [1:16000] 5076 4992 5228 5099 5099 ...
 $ job_admin.                   : num [1:16000] 1 0 1 0 1 1 0 0 1 0 ...
 $ job_blue-collar        

In [113]:
# Make prediction using other SVC model
md_svc <- svm(subscribe ~ ., data=train, kernel='linear', cost=0.01, scale=F)


In [115]:
ypred <- predict(md_svc, X_test)
table(predict=ypred, truth=y_test)

ERROR: Error in table(predict = ypred, truth = y_test): all arguments must have the same length
